# 04 — Feedback: Closed-Loop Learning

This notebook demonstrates **AI Readiness Pillar #4: Feedback**.

You will:
- Simulate a simple model that predicts whether a customer is "high risk" (toy example)
- Capture user feedback / corrections
- Measure performance before/after
- Detect **data drift** with a KS test

> The goal is to show the architecture pattern, not build a perfect model.


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from scipy.stats import ks_2samp

BASE_PATH = Path("..")
DATA_CURATED = BASE_PATH / "data" / "curated"


In [ ]:
customers = pd.read_csv(DATA_CURATED / "customers_silver.csv")
orders = pd.read_csv(DATA_CURATED / "orders_silver.csv")

# Create simple features: order count + total amount per customer
agg = orders.groupby("customer_id").agg(order_count=("order_id","count"), total_amount=("amount","sum")).reset_index()
df = customers.merge(agg, on="customer_id", how="left").fillna({"order_count":0,"total_amount":0})

# Toy label: "high_value" customers (top 20% by total_amount)
threshold = df["total_amount"].quantile(0.8)
df["label_high_value"] = (df["total_amount"] >= threshold).astype(int)

df[["customer_id","order_count","total_amount","label_high_value"]].head()


In [ ]:
X = df[["order_count","total_amount"]]
y = df["label_high_value"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

pred = model.predict(X_test)
print(classification_report(y_test, pred))


## Capture feedback

In real products, feedback comes from:
- user corrections
- outcomes (did the decision help?)
- ticketing / incident reports

We'll simulate feedback where users correct some wrong predictions.


In [ ]:
feedback_path = DATA_CURATED / "feedback.csv"

test = X_test.copy()
test["y_true"] = y_test.values
test["y_pred"] = pred

# Simulate user corrections: pick some false positives/negatives and correct
test["user_confirmed"] = test["y_pred"]
mismatch_idx = test.index[test["y_true"] != test["y_pred"]].tolist()
to_correct = mismatch_idx[:max(5, len(mismatch_idx)//3)]
test.loc[to_correct, "user_confirmed"] = test.loc[to_correct, "y_true"]

feedback = test.loc[to_correct, ["order_count","total_amount","y_true","y_pred","user_confirmed"]].copy()
feedback.to_csv(feedback_path, index=False)

feedback.head(), feedback_path


## Retrain with feedback

Closed-loop means you actually use the corrections to improve the system.


In [ ]:
# Create an updated training set by appending feedback as additional labeled examples
fb = pd.read_csv(feedback_path)
X_fb = fb[["order_count","total_amount"]]
y_fb = fb["user_confirmed"]

X_train2 = pd.concat([X_train, X_fb], ignore_index=True)
y_train2 = pd.concat([y_train, y_fb], ignore_index=True)

model2 = LogisticRegression(max_iter=200)
model2.fit(X_train2, y_train2)

pred2 = model2.predict(X_test)
print("After feedback:")
print(classification_report(y_test, pred2))


## Drift detection (KS test)

When data distributions change, your model becomes unreliable.
We'll compare the distribution of `total_amount` in train vs test.


In [ ]:
stat, p = ks_2samp(X_train["total_amount"], X_test["total_amount"])
print({"ks_stat": float(stat), "p_value": float(p)})

if p < 0.05:
    print("⚠️ Potential drift detected (p < 0.05). Investigate and retrain.")
else:
    print("✅ No strong drift signal detected.")


## Why this matters for AI

Without feedback loops:
- copilots hallucinate and never improve
- agents repeat the same mistakes
- trust declines over time

Feedback turns AI from a demo into a living system.
